In [1]:
import pickle
import random
from math import sqrt
from statistics import mean

import pandas as pd

# Importing Datasets

In [2]:
# World Cup Matches
matches = pd.read_csv("../WorldCupMatches.csv")

# wwmatches
wwmatches = pd.read_csv("../wwmatches.csv")

# Player Stats
stats = pd.read_csv("../PlayersStats.csv")

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (14,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Load `team_players` binary

In [3]:
with open("../../site/server/match_predictor/ml_data/team_players.b", "rb") as f:
    team_players = pickle.load(f)

# Combining the matches

In [4]:
matches = matches.append(wwmatches, sort=True)
matches = matches.reset_index().drop("index", axis=1)

In [5]:
new_matches = pd.DataFrame(columns=matches.columns)

for i in range(len(matches)):
    if matches["Home Team Name"][i] in team_players.keys() and matches["Away Team Name"][i] in team_players.keys():
        new_matches = new_matches.append(matches.iloc[i])

    elif matches["Home Team Name"][i] == "IR Iran" and matches["Away Team Name"][i] in team_players.keys():
        matches["Home Team Name"][i] = "Iran"
        new_matches = new_matches.append(matches.iloc[i])
        
    elif matches["Away Team Name"][i] == "IR Iran" and matches["Home Team Name"][i] in team_players.keys():
        matches["Away Team Name"][i] = "Iran"
        new_matches = new_matches.append(matches.iloc[i])
    
matches = new_matches

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
matches = matches.sort_values("Year")
matches = matches.reset_index().drop("index", axis=1)

In [7]:
len(matches)

3951

In [8]:
matches.head()

,Away Team Goals,Away Team Name,Home Team Goals,Home Team Name,Year
0,1.0,Mexico,4.0,France,1930.0
1,1.0,Spain,0.0,Portugal,1930.0
2,3.0,Poland,0.0,Sweden,1930.0
3,2.0,Sweden,2.0,Belgium,1930.0
4,3.0,Germany,6.0,Denmark,1930.0


## Save matches into CSV file (final.csv)

In [9]:
matches.to_csv("../final.csv", index=0)

# Match players with matches

# Add player's FIFA stats

### Define `final` DataFrame

In [10]:
columns = list(matches.columns)

for i in range(1, 12):
    columns.append("Home Player " + str(i) + " Overall")

for i in range(1, 12):
    columns.append("Away Player " + str(i) + " Overall")

final = pd.DataFrame(columns=columns)

### Preparing utility functions

In [11]:
stats = stats.sort_values(by="Name")
stats = stats.reset_index().drop("index", axis=1)

In [12]:
def clean(x):
    x = x.lower().strip().split(" ")

    if len(x) == 1:
        x = x[0]

    elif len(x) == 2:
        if "." in x[0]:
            x = x[1]
        elif "." in x[1]:
            x = x[0]
        else:
            x = x[0] + x[1]

    elif len(x) == 3:
        if "." in x[0]:
            x = x[1] + x[2]
        elif "." in x[1]:
            x = x[0] + x[2]
        elif "." in x[2]:
            x = x[0] + x[1]
        else:
            x = x[0] + x[1] + x[2]

    elif len(x) == 4:
        if "(" in x[3] or ")" in x[3]:
            if "." in x[0]:
                x = x[1] + x[2]
            elif "." in x[1]:
                x = x[0] + x[2]
            elif "." in x[2]:
                x = x[0] + x[1]
            else:
                x = x[0] + x[1] + x[2]
        else:
            if "." in x[0]:
                x = x[1] + x[2] + x[3]
            elif "." in x[1]:
                x = x[0] + x[2] + x[3]
            elif "." in x[2]:
                x = x[0] + x[1] + x[3]
            elif "." in x[3]:
                x = x[0] + x[1] + x[2]
            else:
                x = x[0] + x[1] + x[2] + x[3]
            
    else:
        pass
    
    return x

def find_start_index(name):
    name = clean(name)
    i = 0
    jump_step = int(sqrt(len(stats)))
    
    while i < len(stats) and name[0] > clean(stats["Name"][i])[0]:
        i += jump_step
    
    if i >= jump_step:
        i -= jump_step

    return i

def find_end_index(name):
    name = clean(name)
    i = len(stats)-1
    jump_step = int(sqrt(len(stats)))
    
    while i >= 0 and name[0] < clean(stats["Name"][i])[0]:
        i -= jump_step

    if i < len(stats)-jump_step:
        i += jump_step

    return i

def find_player_stats(name, debug=False):
    best_match_index = -1
            
    for i in range(find_start_index(name), find_end_index(name)):      
        if name == stats["Name"][i]:
            best_match_index = i
            
    if best_match_index != -1:
        if debug:
            print("Search: ", name, " | ", "Best match: ", stats.iloc[best_match_index]["Name"])
        return stats.iloc[best_match_index]
        
    else:
        return False

def get_nbest_scores(team_name, n=11, debug=False):
    players = team_players[team_name]
    scores = []

    # pprint(players)
    for i in range(len(players)):
        try:
            scores.append(int(players[i][1]))

        except:
            player_stats = find_player_stats(players[i], debug=debug)
            if type(player_stats) is bool:
                try:
                    scores.append(mean(scores))
                    team_players[team_name][i] = [players[i], mean(scores)]
                except:
                    scores.append(60)
                    team_players[team_name][i] = [players[i], 60]
            else:
                player_score = int(player_stats["Overall"])
                scores.append(player_score)
                team_players[team_name][i] = [players[i], player_score]

    scores.sort()
    return scores[:n]

### Define `merging`

In [13]:
def run_merging(debug=False):
    global final
    
    for i in range(len(matches)):
        home_team_name = matches["Home Team Name"][i]
        away_team_name = matches["Away Team Name"][i]

        # # Add player's FIFA Overall scores # #
        home_players_stats = get_nbest_scores(home_team_name, debug=debug)
        away_players_stats = get_nbest_scores(away_team_name, debug=debug)

        # # Agregate data # #
        data = list(matches.iloc[i])
        data += home_players_stats
        data += away_players_stats

        # # Append current data # # 
        final = final.append(pd.Series(data, index=final.columns), ignore_index=True)

### Run `merging` (will take about an hour)

In [14]:
run_merging(debug=False)

In [15]:
final.head()

,Away Team Goals,Away Team Name,Home Team Goals,Home Team Name,Year,Home Player 1 Overall,Home Player 2 Overall,Home Player 3 Overall,Home Player 4 Overall,Home Player 5 Overall,...,Away Player 2 Overall,Away Player 3 Overall,Away Player 4 Overall,Away Player 5 Overall,Away Player 6 Overall,Away Player 7 Overall,Away Player 8 Overall,Away Player 9 Overall,Away Player 10 Overall,Away Player 11 Overall
0,1.0,Mexico,4.0,France,1930.0,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
1,0.0,France,1.0,Argentina,1930.0,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
2,0.0,Peru,1.0,Uruguay,1930.0,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
3,3.0,Mexico,6.0,Argentina,1930.0,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
4,2.0,Argentina,4.0,Uruguay,1930.0,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60


# Save `final` into CSV file and `team_players` into binary file

In [16]:
final.to_csv("../final.csv", index=0)

In [17]:
with open("../../site/server/match_predictor/ml_data/team_players.b", "wb") as f:
    pickle.dump(team_players, f)